# Challenge 4 study - Quantized Model with 2 channels Conv Input Layer

## Import libraries and data, init virtual device

In [1]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, ReLU, Conv2D, Flatten, Dropout

from akida import Device, AKD1000
import cnn2snn

device = AKD1000()

2022-11-11 13:31:05.238483: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-11 13:31:05.437183: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-11 13:31:05.437258: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-11 13:31:05.473129: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-11 13:31:06.432137: W tensorflow/stream_executor/platform/de

## Data Preparation

In [2]:
coords_ch4 = np.load('coords_ch4.npy')
color = np.load('result_ch4.npy')

# Pack in a dataframe and scale for 8-bits quantization (first layer will be quantized on 8-bits)
data = pd.DataFrame(coords_ch4, columns = ['x', 'y'])
data = round(255*data).astype(np.uint8)


# Shuffle and split train/test
data['color'] = color
data_train = data.sample(frac=0.7, axis=0)
data_test = data.drop(data_train.index)

# Split x y
x_train = data_train.drop('color', axis=1)
y_train = data_train['color']
x_test = data_test.drop('color', axis=1)
y_test = data_test['color']


# Attempt to encode coordinates as an image: put the x value as grey scale on the left, then y on the right
x = np.empty((5, 5, 1))
y = np.empty((5, 5, 1))
z = np.empty((5, 5, 1))

x_train_coded = np.empty((7000, 5, 5, 3))
for i, p in enumerate(x_train.to_numpy()):
    x.fill(p[0])
    y.fill(p[1])
    z.fill(0)
    x_train_coded[i] = np.concatenate((x, y, z), axis=2)
x_train_coded = x_train_coded.astype(np.uint8)

x_test_coded = np.empty((3000, 5, 5, 3))
for i, p in enumerate(x_test.to_numpy()):
    x.fill(p[0])
    y.fill(p[1])
    z.fill(0)
    x_test_coded[i] = np.concatenate((x, y, z), axis=2)
x_test_coded = x_test_coded.astype(np.uint8)

x_train_coded[0]

array([[[238,  86,   0],
        [238,  86,   0],
        [238,  86,   0],
        [238,  86,   0],
        [238,  86,   0]],

       [[238,  86,   0],
        [238,  86,   0],
        [238,  86,   0],
        [238,  86,   0],
        [238,  86,   0]],

       [[238,  86,   0],
        [238,  86,   0],
        [238,  86,   0],
        [238,  86,   0],
        [238,  86,   0]],

       [[238,  86,   0],
        [238,  86,   0],
        [238,  86,   0],
        [238,  86,   0],
        [238,  86,   0]],

       [[238,  86,   0],
        [238,  86,   0],
        [238,  86,   0],
        [238,  86,   0],
        [238,  86,   0]]], dtype=uint8)

## Model creation with Keras

In [13]:
base = Sequential(name="Base_ch4")
base.add(Conv2D(1, (3, 3), input_shape=(5, 5, 3), name='C2D'))
base.add(ReLU(name='relu0'))
#base.add(Dropout(0.2))
base.add(Flatten())
base.add(Dense(1000, name='FC1'))
base.add(ReLU(name='relu1'))
base.add(Dense(500, name='FC2'))
base.add(ReLU(name='relu2'))
base.add(Dense(250, name='FC3'))
base.add(ReLU(name='relu3'))
base.add(Dense(100, name='FC4'))
base.add(ReLU(name='relu4'))
base.add(Dense(50, name='FC5'))
base.add(ReLU(name='relu5'))
base.add(Dense(1, name='FC6'))
base.add(Activation('sigmoid', name='sigmoid'))

#base.summary()

## Model quantization (4-bits, input: 8-bit)

In [14]:
qmodel = cnn2snn.quantize(base, weight_quantization=4, activ_quantization=4, input_weight_quantization=8)
qmodel.summary()

print('\nAkida Compatibility:', cnn2snn.check_model_compatibility(qmodel, input_is_image=True))

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 C2D (QuantizedConv2D)       (None, 3, 3, 1)           28        
                                                                 
 relu0 (QuantizedReLU)       (None, 3, 3, 1)           0         
                                                                 
 flatten_3 (Flatten)         (None, 9)                 0         
                                                                 
 FC1 (QuantizedDense)        (None, 1000)              10000     
                                                                 
 relu1 (QuantizedReLU)       (None, 1000)              0         
                                                                 
 FC2 (QuantizedDense)        (None, 500)               500500    
                                                                 
 relu2 (QuantizedReLU)       (None, 500)             

## Preview akida model

In [15]:
akmodel = cnn2snn.convert(qmodel, input_is_image=True)
akmodel.map(device)
akmodel.summary()

                   Model Summary                   
___________________________________________________
Input shape  Output shape  Sequences  Layers  NPs
[5, 5, 3]    [1, 1, 1]     1          7       6  
___________________________________________________

________________________________________________________
Layer (type)      Output shape  Kernel shape       NPs

====== HW/C2D-FC6 (Hardware) - size: 560632 bytes ======

C2D (InputConv.)  [3, 3, 1]     (3, 3, 3, 1)       N/A
________________________________________________________
FC1 (Fully.)      [1, 1, 1000]  (1, 1, 9, 1000)    1  
________________________________________________________
FC2 (Fully.)      [1, 1, 500]   (1, 1, 1000, 500)  1  
________________________________________________________
FC3 (Fully.)      [1, 1, 250]   (1, 1, 500, 250)   1  
________________________________________________________
FC4 (Fully.)      [1, 1, 100]   (1, 1, 250, 100)   1  
________________________________________________________
FC5 (Fully.)

## Train quantized model

In [16]:
qmodel.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
            )
qmodel.fit(x_train_coded, y_train, batch_size=64, epochs=600)
qmodel.evaluate(x_test_coded, y_test)

Epoch 1/600
110/110 [==============================] - 6s 28ms/step - loss: 0.6932 - accuracy: 0.5009
Epoch 2/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.5023
Epoch 3/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.5023
Epoch 4/600
110/110 [==============================] - 3s 27ms/step - loss: 0.6932 - accuracy: 0.5023
Epoch 5/600
110/110 [==============================] - 3s 27ms/step - loss: 0.6932 - accuracy: 0.5023
Epoch 6/600
110/110 [==============================] - 3s 27ms/step - loss: 0.6932 - accuracy: 0.5023
Epoch 7/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.5023
Epoch 8/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.5023
Epoch 9/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.5023
Epoch 10/600
110/110 [==============================] - 3s 31ms/step - loss: 0.693

KeyboardInterrupt: 